In [1]:
"""
Title: AI539 - Machine Learning Challenge 2022 Winter Final Project
Authur: Yun-Yi Tseng
Date: 2022/03/13

This notebook conduct 9 experiments for 3 challenges

"""
import numpy as np
import pandas as pd
import random as rn

import matplotlib.pyplot as plt
import os
from tqdm import tqdm

plt.style.use("ggplot")
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# set seed for experiments
SEED = 539
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)

In [2]:
# This section is for running on colab. See Readme.md. 

# from google.colab import drive
# drive.mount('/content/drive')

# df = pd.read_csv(
#     "/content/drive/MyDrive/air_quality_2018to2020.csv", index_col="time", parse_dates=True
# )
# geo = pd.read_csv("/content/drive/MyDrive/geodata.csv")

In [3]:
df = pd.read_csv(
    "../data/processed/air_quality_2018to2020.csv", index_col="time", parse_dates=True
)
# geo data for challenge 3
geo = pd.read_csv("../data/external/geodata.csv")

In [4]:
df.head()

,station,longitude,latitude,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,CH4,NMHC,THC,PM2.5
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,Sanyi,120.758833,24.382942,11.6,0.26,0.8,4.7,5.5,39.6,53,0,92,2.3,24,24,3.9,3.8,NaN,NaN,NaN,21
2018-01-01 01:00:00,Sanyi,120.758833,24.382942,11.6,0.28,0.9,2.3,3.2,40.1,47,0,93,0.5,24,23,3.7,3.8,NaN,NaN,NaN,22
2018-01-01 02:00:00,Sanyi,120.758833,24.382942,11.4,0.27,0.3,2.6,3,39.5,46,0,94,0.5,25,23,4.3,3.9,NaN,NaN,NaN,20
2018-01-01 03:00:00,Sanyi,120.758833,24.382942,11.2,0.26,0.3,2.6,2.9,39.5,46,0,95,0.5,24,21,3.4,3.6,NaN,NaN,NaN,22
2018-01-01 04:00:00,Sanyi,120.758833,24.382942,11.1,0.25,0.6,2.2,2.8,39.6,41,0,96,0.5,22,23,3.5,3.5,NaN,NaN,NaN,20


In [5]:
df.shape

(2023128, 21)

In [6]:
numerical_columns = [
    "AMB_TEMP",
    "CH4",
    "CO",
    "NMHC",
    "NO",
    "NO2",
    "NOx",
    "O3",
    "PM10",
    "PM2.5",
    "RAINFALL",
    "RH",
    "SO2",
    "THC",
    "WD_HR",
    "WIND_DIREC",
    "WIND_SPEED",
    "WS_HR",
]

# drop missing labels and remove non-numeric values
# #: invalid values detected by tools; *: invalid values detected by program; x: invalid values detected by humans
df.dropna(how='any',axis=0,subset=['PM2.5'],inplace=True)

for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Chanllenge 1: Missing values

strategies:
- fill missing values by mean of training data
- Drop THC, CH4 and NMHC and fill other columns with the last valid value
- Drop THC, CH4 and NMHC and fill other columns with mean

In [7]:
# Preprocessing for the experiments
def preprocessing(df, grouped=False, category_cols= []):
    
    # time_step for lstm
    time_step = 1
    
    
    if not grouped:
        df = df.groupby(["station", pd.Grouper(freq="D")]).mean()
    
    # shift labels for predictions
    for station, d in df.groupby(["station"]):
        df.loc[station, "target"] = d["PM2.5"].shift(periods=-1).values
    
    # for every station drop the first values
    df.dropna(how='any',axis=0,inplace=True)
    
    # encode rainfall if it is out of Q3 + 1.5 x IQR
    Q1 = df.loc[df.index.get_level_values('time').year==2018, "RAINFALL"].quantile(0.25)
    Q3 = df.loc[df.index.get_level_values('time').year==2018, "RAINFALL"].quantile(0.75)
    IQR = Q3 - Q1

    df["RAINFALL"] = np.where(df["RAINFALL"] < (Q3 + 1.5 * IQR), 1, 0)
    
    # Normalization of features
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(
        df.loc[df.index.get_level_values('time').year==2018, :]
        .drop(["longitude", "latitude", "RAINFALL",'PM2.5',"target"]+category_cols, axis=1)
        .values
    )

    df[
        df.drop(["longitude", "latitude", "RAINFALL",'PM2.5',"target"]+category_cols, axis=1).columns
    ] = scaler.transform(df.drop(["longitude", "latitude", "RAINFALL",'PM2.5',"target"]+category_cols, axis=1).values)

    # Normalization of labels
    label_scaler = MinMaxScaler(feature_range=(-1,1))
    label_scaler.fit(df.loc[df.index.get_level_values('time').year==2018, "target"].values.reshape(-1, 1))
    df['target'] = label_scaler.transform(df['target'].values.reshape(-1, 1))
    
    train_X, train_Y = [], []
    validation_X, validation_Y = [], []
    test_X, test_Y = [], []

    for station, d in df.groupby('station'):
        # find the index of first day and last day of each years
        first_train_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2018].index[0])
        last_train_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2018].index[-1])
        first_val_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2019].index[0])
        last_val_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2019].index[-1])
        first_test_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2020].index[0])
        last_test_date = d.index.get_loc(d.loc[d.index.get_level_values('time').year==2020].index[-1])
        
        # append previous time step values to fit lstm input format
        for i in range(first_train_date + time_step, last_train_date+1):
            indices = range(i - time_step, i, 1)
            train_X.append(d.reset_index(drop=True).loc[indices].drop(['PM2.5','target'],axis=1).values)
            train_Y.append(d.reset_index(drop=True).loc[i-1,'target'])
        for i in range(first_val_date + time_step, last_val_date):
            indices = range(i - time_step, i, 1)
            validation_X.append(d.reset_index(drop=True).loc[indices].drop(['PM2.5','target'],axis=1).values)
            validation_Y.append(d.reset_index(drop=True).loc[i-1,'target'])
        for i in range(first_test_date + time_step, last_test_date):
            indices = range(i - time_step, i, 1)
            test_X.append(d.reset_index(drop=True).loc[indices].drop(['PM2.5','target'],axis=1))
            test_Y.append(d.reset_index(drop=True).loc[i-1,'target'])
    
    return np.array(train_X), np.array(train_Y), np.array(validation_X), np.array(validation_Y), np.array(test_X), np.array(test_Y), label_scaler # return label scaler for recover result

In [8]:
def train_lstm(train_X, train_Y, validation_X, validation_Y):

    # building models
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='Adam')
    
    # if out of memory, lower the batch_size
    history = model.fit(train_X, train_Y, epochs=200, batch_size=256, validation_data=(validation_X, validation_Y), verbose=2, shuffle=True)
    
    return model, history

In [9]:
# helper function for plot experiment result
def plot_result(X, True_Y, model, title):

    pred = model.predict(X)
    
    pred = label_scaler.inverse_transform(pred.reshape(-1,1))
    True_Y = label_scaler.inverse_transform(True_Y.reshape(-1,1))

    plt.figure(figsize=(15,12))
    plt.plot(pred[:200], label='prediction')
    plt.plot(True_Y[:200], label='True label')
    plt.title(title)
    plt.legend()
    plt.show()
    
    return mean_absolute_error(True_Y, pred)

In [10]:
# helper function for plot training loss
def plot_loss(history, title):
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title(title)
    plt.legend()
    plt.show()

In [11]:
# Method 1: fill missing values by mean of training data
df1 = df.copy()

for col in numerical_columns:
    train_mean = df1.loc["2018", col].mean()
    df1.fillna(train_mean,inplace=True)

(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df1)


In [12]:
# training
model, history = train_lstm(train_X,train_Y,validation_X,validation_Y)

Epoch 1/200
110/110 - 4s - loss: 0.2185 - val_loss: 0.0480 - 4s/epoch - 38ms/step
Epoch 2/200
110/110 - 1s - loss: 0.0508 - val_loss: 0.0447 - 611ms/epoch - 6ms/step
Epoch 3/200
110/110 - 1s - loss: 0.0475 - val_loss: 0.0413 - 630ms/epoch - 6ms/step
Epoch 4/200
110/110 - 1s - loss: 0.0457 - val_loss: 0.0402 - 616ms/epoch - 6ms/step
Epoch 5/200
110/110 - 1s - loss: 0.0448 - val_loss: 0.0389 - 604ms/epoch - 5ms/step
Epoch 6/200
110/110 - 1s - loss: 0.0445 - val_loss: 0.0388 - 598ms/epoch - 5ms/step
Epoch 7/200
110/110 - 1s - loss: 0.0445 - val_loss: 0.0385 - 628ms/epoch - 6ms/step
Epoch 8/200
110/110 - 1s - loss: 0.0440 - val_loss: 0.0388 - 610ms/epoch - 6ms/step
Epoch 9/200
110/110 - 1s - loss: 0.0439 - val_loss: 0.0387 - 601ms/epoch - 5ms/step
Epoch 10/200
110/110 - 1s - loss: 0.0439 - val_loss: 0.0382 - 599ms/epoch - 5ms/step
Epoch 11/200
110/110 - 1s - loss: 0.0437 - val_loss: 0.0385 - 623ms/epoch - 6ms/step
Epoch 12/200
110/110 - 1s - loss: 0.0433 - val_loss: 0.0379 - 613ms/epoch - 

KeyboardInterrupt: 

In [ ]:
plot_loss(history, 'fill_by_mean_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'fill_by_mean_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'fill_by_mean_testset')

In [ ]:
result = pd.DataFrame({},columns=['Validation_MAE','test_MAE'])
result.loc['fill_by_mean'] = [valid_mae,test_mae]

In [ ]:
result

In [ ]:
# Method 2: fill missing values by last valid values
df2 = df.copy()

df2.drop(['THC','CH4','NMHC'],axis=1,inplace=True)
for station, d in df2.groupby('station'):
    d.fillna(method='ffill',inplace=True)


(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df2)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'drop_and_ffill_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'drop_and_ffill_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'drop_and_ffill_test')

In [ ]:
result.loc['drop_and_ffill'] = [valid_mae,test_mae]

In [ ]:
result

In [ ]:
# Method 3: drop THC, CH4, NMHC and fill by mean of training data
df3 = df.copy()

df3.drop(['THC','CH4','NMHC'],axis=1,inplace=True)
for col in numerical_columns:
    if col in df3.columns:
        train_mean = df3.loc["2018", col].mean()
        df3.fillna(train_mean,inplace=True)


(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df3)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'drop_and_fill_by_mean_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'drop_and_fill_by_mean_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'drop_and_fill_by_mean_test')

In [ ]:
result.loc['drop_and_fill_by_mean'] = [valid_mae,test_mae]

In [ ]:
result

In [ ]:
df4 = df.copy()
df4 = df4.groupby(["station", pd.Grouper(freq="D")]).mean()
for station, d in df4.groupby(["station"]):
    # df4.loc[station, "previous"] = d.loc[station].groupby([d.loc[station].index.month,d.loc[station].index.day])['PM2.5'].shift().values
    df4.loc[station, 'previous'] = d.loc[station,"PM2.5"].shift().values
    
df4_2019 = df4.loc[df4.index.get_level_values('time').year==2019].dropna(how='any',subset=['PM2.5','previous'])

valid_mae = mean_absolute_error(df4_2019['PM2.5'],df4_2019['previous'])

df4_2020 = df4.loc[df4.index.get_level_values('time').year==2020].dropna(how='any',subset=['PM2.5','previous'])

test_mae = mean_absolute_error(df4_2020['PM2.5'],df4_2020['previous'])

In [ ]:
result.loc['previous_day'] = [valid_mae,test_mae]

In [ ]:
result.style.highlight_min(color="green", axis=0)

# Chanllenge 2: Temporal data representation

strategies:
- Add new columns of year, month, day.
- Add new features of previous 7 days target.
- Add new features that represent the statistics of last week

In [ ]:
def fill_na(df):
    for col in numerical_columns:
        train_mean = df.loc["2018", col].mean()
        df.fillna(train_mean,inplace=True)

    return df

In [ ]:
# Method 1: Add new columns of year, month, day.
df1 = df.copy()

df1 = fill_na(df1)

df1['year'] = df1.index.year
df1['month'] = df1.index.month
df1['day'] = df1.index.day

(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df1)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'add_time_columns_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'add_time_columns_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'add_time_columns_test')

In [ ]:
temporal_result = pd.DataFrame({},columns=['Validation_MAE','test_MAE'])
temporal_result.loc['add_time_columns'] = [valid_mae,test_mae]

In [ ]:
temporal_result

In [ ]:
# Method 2: add new features of previous 7 days target.
df2 = df.copy()

df2 = fill_na(df2)

df2 = df2.groupby(["station", pd.Grouper(freq="D")]).mean()

for t in range(7):
    df2[f't-{t+1}'] = df2['PM2.5'].shift(periods=t+1)
df2.fillna(0,inplace=True)
    
(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df2,grouped=True)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'add_prev_t_columns_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'add_prev_t_columns_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'add_prev_t_columns_test')

In [ ]:
temporal_result.loc['add_prev_t_columns'] = [valid_mae,test_mae]

In [ ]:
temporal_result

In [ ]:
# Method 3: add new features that represent the statistics of last week
df3 = df.copy()

df3 = fill_na(df3)

df3 = df3.groupby(["station", pd.Grouper(freq="D")]).mean()

df3['last_week_mean'] = df3['PM2.5'].rolling(7).mean()
df3['last_week_min'] = df3['PM2.5'].rolling(7).min()
df3['last_week_max'] = df3['PM2.5'].rolling(7).max()
df3['diff'] = df3['PM2.5'].diff(periods=1)
df3['last_week_diff_mean'] = df3['diff'].rolling(7).mean()
df3['last_week_diff_min'] = df3['diff'].rolling(7).min()
df3['last_week_diff_max'] = df3['diff'].rolling(7).max()

df3.fillna(0)
    
(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df3,grouped=True)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'add_last_week_statistics_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'add_last_week_statistics_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'add_last_week_statistics_test')

In [ ]:
temporal_result.loc['add_last_week_statistics'] = [valid_mae,test_mae]

In [ ]:
temporal_result

In [ ]:
temporal_result.loc['drop_and_fill_by_mean'] = result.loc['drop_and_fill_by_mean']
temporal_result.loc['previous_day'] = result.loc['previous_day']

In [ ]:
temporal_result.style.highlight_min(color="green", axis=0)

# Chanllenge 3: Spatial data representation

Strategies:
- Use kmeans to separate into 5 groups
- Apply one hot representation to counties(22 counties)
- Factorize county to numeric representation

In [ ]:
# Method 1: Use kmeans to separate into 5 groups
df1 = df.copy()

df1 = fill_na(df1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(df1[['longitude','latitude']].values)
df1['geo_group'] = kmeans.labels_
    
(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df1,category_cols=['geo_group'])

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'kmeans_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'kmeans_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'kmeans_test')

In [ ]:
spatial_result = pd.DataFrame({},columns=['Validation_MAE','test_MAE'])
spatial_result.loc['kmeans'] = [valid_mae,test_mae]

In [ ]:
spatial_result

In [ ]:
# Method 2: Apply one hot representation to counties(22 counties)
df2 = df.copy()

df2 = fill_na(df2)

new_geo = geo.copy()


df2['county'] = pd.merge(df2,new_geo, left_on=  ['station'],
                   right_on= ['siteengname'], 
                   how = 'left')['county'].values

df2 = pd.concat([df2.drop('county',axis=1), pd.get_dummies(df2['county'])], axis=1)
    
(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df2)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'one_hot_county_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'one_hot_county_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'one_hot_county_test')

In [ ]:
spatial_result.loc['one_hot_county'] = [valid_mae,test_mae]

In [ ]:
spatial_result

In [ ]:
# Method 3: Factorize county to numeric representation
df3 = df.copy()

df3 = fill_na(df3)

new_geo = geo.copy()

df3['county'] = pd.merge(df3,geo, left_on=  ['station'],
                   right_on= ['siteengname'], 
                   how = 'left')['county'].values

df3['county'] = pd.factorize(df3['county'])[0]
    
(
    train_X,
    train_Y,
    validation_X,
    validation_Y,
    test_X,
    test_Y,
    label_scaler
) = preprocessing(df3)

In [ ]:
model,history = train_lstm(train_X,train_Y,validation_X,validation_Y)

In [ ]:
plot_loss(history, 'categorize_county_loss')

In [ ]:
valid_mae = plot_result(validation_X, validation_Y, model, 'categorize_county_validation')

In [ ]:
test_mae = plot_result(test_X, test_Y, model, 'categorize_county_test')

In [ ]:
spatial_result.loc['categorize_county'] = [valid_mae,test_mae]
spatial_result.loc['drop_and_fill_by_mean'] = result.loc['drop_and_fill_by_mean']
spatial_result.loc['previous_day'] = result.loc['previous_day']

In [ ]:
spatial_result.style.highlight_min(color="green", axis=0)